In [1]:
import numpy as np
import gurobipy as gp

In [2]:
count_units = 3
count_times = 6

costs_startup_const = [800, 500, 250]
costs_generation = [5, 15, 30]
power_outputs_minimum = [80, 50, 30]
power_outputs_maximum = [300, 200, 100]
startups = [100, 70, 40]
startdowns = [80, 50, 30]
rampups = [50, 60, 70]
rampdowns = [30, 40, 50]
minimumups = [3, 2, 1]
minimumdowns = [2, 2, 2]
commitments_initial = [1, 0, 0]
power_outputs_initial = [120, 0, 0]
minimumups_initial  = [2, 0, 0]
minimumdowns_initial  = [0, 0, 0]
demands = [220, 250, 200, 170, 230, 190]
reserves = [10, 10, 10, 10, 10, 10]

In [3]:
model = gp.Model()
model.setParam("OutputFlag", 0)

Set parameter Username
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01


In [4]:
commitments = model.addVars(range(count_units), range(count_times), vtype=gp.GRB.BINARY)
power_outputs = model.addVars(
    range(count_units), range(count_times), lb=0,
    ub=np.tile(np.array(power_outputs_maximum)[:, None], (1, count_times))
)
power_outputs_bars = model.addVars(
    range(count_units), range(count_times), lb=0,
    ub=np.tile(np.array(power_outputs_maximum)[:, None], (1, count_times))
)
costs_startup = model.addVars(
    range(count_units), range(count_times), lb=0)

In [5]:
model.addConstrs(
    power_outputs[i, t] >= power_outputs_minimum[i] * commitments[i, t]
    for i in range(count_units) for t in range(count_times)
)

model.addConstrs(
    power_outputs[i, t] <= power_outputs_maximum[i] * commitments[i, t]
    for i in range(count_units) for t in range(count_times)
)

model.addConstrs(
    power_outputs[i, t] <= power_outputs_bars[i, t]
    for i in range(count_units) for t in range(count_times)
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>}

In [6]:
model.addConstrs(
    gp.quicksum(power_outputs[i, t] for i in range(count_units)) == demands[t]
    for t in range(count_times)
)

model.addConstrs(
    gp.quicksum(power_outputs_bars[i, t] for i in range(count_units)) >= demands[t] + reserves[t]
    for t in range(count_times)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [7]:
#
# t = 0 -> costs_startup lb
model.addConstrs(
    costs_startup[i, t] >= costs_startup_const[i] * (
        commitments[i, t] - gp.quicksum(commitments[i, t-n] for n in range(tau))
    )
    for i in range(count_units) for t in range(1, count_times) for tau in range(t)
) 

{(0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 0): <gurobi.Constr *Awaiting Model Upd

In [8]:
#
model.addConstrs(
    power_outputs_bars[i, t] <= (
        power_outputs[i, t-1]
        + rampups[i] * commitments[i, t-1]
        + startups[i] * (commitments[i, t] - commitments[i, t-1])
        + power_outputs_maximum[i] * (1 - commitments[i, t])
    )
    for i in range(count_units) for t in range(1, count_times)
)

model.addConstrs(
    power_outputs_bars[i, 0] <= (
        power_outputs_initial[i] 
        + rampups[i] * commitments_initial[i]
        + startups[i] * (commitments[i, 0] - commitments_initial[i])
        + power_outputs_maximum[i] * (1 - commitments[i, 0])
    )
    for i in range(count_units)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [9]:
model.addConstrs(
    power_outputs_bars[i, t] <= (
        power_outputs_maximum[i] * commitments[i, t+1]
        + startdowns[i] * (commitments[i, t] - commitments[i, t+1])
    )
    for i in range(count_units) for t in range(count_times - 1)
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

In [10]:
model.addConstrs(
    power_outputs[i, t-1] - power_outputs[i, t] <= (
        rampdowns[i] * commitments[i, t] 
        + startdowns[i] * (commitments[i, t-1] - commitments[i, t])
        +  power_outputs_maximum[i] * (1 - commitments[i, t-1])
    )
    for i in range(count_units) for t in range(1, count_times)
)

model.addConstrs(
    power_outputs_initial[i] - power_outputs[i, 0] <= (
        rampdowns[i] * commitments_initial[i] 
        + startdowns[i] * (commitments_initial[i] - commitments[i, 0])
        +  power_outputs_maximum[i] * (1 - commitments_initial[i])
    )
    for i in range(count_units)
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}